In [52]:
import pandas as pd
import pyreadr

result = pyreadr.read_r("5v_cleandf.rdata") 

df = result['df']

df.head()

,dep_name,esi,age,gender,ethnicity,race,lang,religion,maritalstatus,employstatus,...,cc_vaginaldischarge,cc_vaginalpain,cc_weakness,cc_wheezing,cc_withdrawal-alcohol,cc_woundcheck,cc_woundinfection,cc_woundre-evaluation,cc_wristinjury,cc_wristpain
0,B,4,40,Male,Hispanic or Latino,White or Caucasian,English,None,Single,Full Time,...,0,0,0,0,0,0,0,0,0,0
1,B,4,66,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed,...,0,0,0,0,0,0,0,0,0,0
2,B,2,66,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed,...,0,0,0,0,0,0,0,0,0,0
3,A,2,66,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed,...,0,0,0,0,0,0,0,0,0,0
4,A,3,84,Female,Hispanic or Latino,Other,Other,Pentecostal,Widowed,Retired,...,0,0,0,0,0,0,0,0,0,0


In [42]:
#df.to_csv("Hospital Triage and Patient History.csv",index=False)

In [43]:

# The data will undergo a cleaning process to make it suitable for binary classification.
# Binary classification will help us determine whether it is an emergency or if the patient can be relocated.

In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

significantColumns = ['cc_chestpain', 'cc_breathingdifficulty', 'cc_syncope', 'cc_unresponsive', 'cc_seizure-newonset', 'cc_seizure-priorhxof', 'cc_seizures',
                            'cc_bleeding/bruising', 'cc_hyperglycemia', 'cc_hypertension', 'cc_hypotension', 'cc_strokealert', 'cc_overdose-accidental',
                            'cc_overdose-intentional', 'cc_suicidal']

def numberOfIncompletaData(columns, dataFrame):
    # Crear una columna indicando si hay datos inconclusos en las columnas determinantes de emergencia
    dataFrame['incomplete_data'] = dataFrame[columns].isnull().all(axis=1)

    # Revisamos la cantidad de verdaderos y falsos para detectar errores
    return dataFrame['incomplete_data'].value_counts()


In [58]:
# Para comprobar si se corrigieron todos los datos faltantes, detectamos la cantidad de tipos de valores iniciales
print(numberOfIncompletaData(significantColumns, df))

# Manejo de valores nulos en las columnas seleccionadas, utilizando el valor más frecuente para rellenar los valores
imputer = SimpleImputer(strategy='most_frequent')

df[significantColumns] = imputer.fit_transform(df[significantColumns])

# Y por último, revisamos que ya no queden valores nulos en las columnas significativas
print(numberOfIncompletaData(significantColumns, df))

False    557901
True       2585
Name: incomplete_data, dtype: int64
False    560486
Name: incomplete_data, dtype: int64


In [60]:
df.head()

# Codificación de variables categóricas
df_encoded = pd.get_dummies(df, columns=significantColumns)

# # Reemplazar la etiqueta de las filas con datos inconclusos en cualquier columna como 0
# df_encoded = df_encoded.fillna(0)
df_encoded.head()

,dep_name,esi,age,gender,ethnicity,race,lang,religion,maritalstatus,employstatus,...,cc_vaginalpain,cc_weakness,cc_wheezing,cc_withdrawal-alcohol,cc_woundcheck,cc_woundinfection,cc_woundre-evaluation,cc_wristinjury,cc_wristpain,incomplete_data
0,B,4,40,Male,Hispanic or Latino,White or Caucasian,English,None,Single,Full Time,...,0,0,0,0,0,0,0,0,0,False
1,B,4,66,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed,...,0,0,0,0,0,0,0,0,0,False
2,B,2,66,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed,...,0,0,0,0,0,0,0,0,0,False
3,A,2,66,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed,...,0,0,0,0,0,0,0,0,0,False
4,A,3,84,Female,Hispanic or Latino,Other,Other,Pentecostal,Widowed,Retired,...,0,0,0,0,0,0,0,0,0,False


In [ ]:
# División de datos
X = df_encoded.drop('emergency_flag_column', axis=1)  # Ajusta la columna objetivo
y = df_encoded['emergency_flag_column']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Validación del modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')